In [1]:
import os
import subprocess
import json

In [2]:
def compile_java(file_path, class_path, log_folder_success, log_folder_fail, output_folder):
    r_errors_count = 0
    gen_r_errors_count = 0
    try:
        successful_compile = False
        # If the output directory doesn't exist, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        # Compile the .java file with the provided class_path and specify the output directory for .class files
        result = subprocess.run(['javac', '-cp', class_path, '-d', output_folder, file_path], capture_output=True, text=True, timeout=10)

        if result.returncode == 0:
            print(f"Successfully compiled \"{file_path}\".")
            stderr_log = result.stderr
            successful_compile = True
        else:
            warning_val = False
            num_errors = 1
            stderr_lines = result.stderr.splitlines()
            num_errors_line = stderr_lines[-1]
            try:
                num_errors = int(num_errors_line.split()[0])  # Extract the number of error(s)
            except ValueError:
                warning_val = True

            r_errors_count = result.stderr.count("error: package R does not exist")
            gen_r_errors_count = result.stderr.count("error: package gen does not exist")
            total_r_errors_count = r_errors_count + gen_r_errors_count

            # Check if all errors are related to "package R does not exist"
            if num_errors == total_r_errors_count and warning_val == False:
                print(f"Successfully compiled, ignoring {num_errors} 'package R does not exist' errors for \"{file_path}\".")
                stderr_log = "No error (Ignored 'package R does not exist' errors)"
                successful_compile = True
            else:
                if warning_val:
                    print(f"Compiled with warning.")
                    stderr_log = "Warning present"
                    successful_compile = True
                else:
                    print(f"Failed to compile \"{file_path}\".")
                    stderr_log = result.stderr
                    successful_compile = False

        # Read the content of the Java file
        with open(file_path, 'r', errors='ignore') as java_file:
            java_code = java_file.read()

        # Prepare the JSON log
        log_data = {
            "file": file_path,
            "java_code": java_code,
            "stdout": result.stdout,
            "stderr": stderr_log,
            "return_code": result.returncode
        }
        
        # Save the logs to a file named after the Java file being compiled
        log_file_name = os.path.basename(file_path).replace('.java', '.json')
        if successful_compile:
            log_file_path = os.path.join(log_folder_success, log_file_name)
        else:
            log_file_path = os.path.join(log_folder_fail, log_file_name)
        
        with open(log_file_path, 'w') as log_file:
            json.dump(log_data, log_file, indent=4)

        return successful_compile

    except subprocess.TimeoutExpired:
        print(f"Compilation of {file_path} timed out.")
        return False

In [3]:
def calculate_success_rate(directory, class_path, log_folder_success, log_folder_fail, output_folder):
    if not os.path.exists(log_folder_success):
        os.makedirs(log_folder_success)

    if not os.path.exists(log_folder_fail):
        os.makedirs(log_folder_fail)

    java_files = [f for f in os.listdir(directory) if f.endswith('.java')]
    if not java_files:
        print("No .java files found.")
        return 0

    total_files = len(java_files)
    successful_compilations = 0

    for java_file in java_files:
        file_path = os.path.join(directory, java_file)
        if compile_java(file_path, class_path, log_folder_success, log_folder_fail, output_folder):
            successful_compilations += 1

    success_rate = (successful_compilations / total_files) * 100
    return success_rate, successful_compilations

In [4]:
# For Zero-shot without Self-Consistency Temp 0.5

dir_type = "zero-shot"
dir_path = f"/home/azmain/code_for_compilation_test/{dir_type}-temp-0.5/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/temp_0.5_no_self_c/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/temp_0.5_no_self_c/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/{dir_type}_temp_0.5_no_self_c/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/xstream_class_25.java".
Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/hibernate_class_24.java".
Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/hibernate_class_19.java".
Failed to compile "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/hibernate_class_44.java".
Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/JodaTime29.java".
Successfully compiled, ignoring 1 'package R does not exist' errors for "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/Android02.java".
Failed to compile "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/xstream_class_23.java".
Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/xstream_class_3.java".
Successfully compiled "/home/azmain/code_for_compilation_test/zero-shot-temp-0.5/JodaTime08.java".
Successfully compiled "/hom

In [4]:
# For Zero-shot Self-Consistency from 10 Sample

dir_type = "zero-shot"
dir_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/{dir_type}-compiled/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/xstream_class_25.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/hibernate_class_24.java".
Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/hibernate_class_19.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/hibernate_class_44.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/JodaTime29.java".
Successfully compiled, ignoring 1 'package R does not exist' errors for "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-zero-shot/Android02.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_wit

In [4]:
# For Zero-shot Self-Consistency from 20 Sample

dir_type = "from-20-sample"
dir_path = f"/home/azmain/code_for_compilation_test/{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/zero-shot-{dir_type}/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/xstream_class_25.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/hibernate_class_24.java".
Failed to compile "/home/azmain/code_for_compilation_test/from-20-sample/hibernate_class_19.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/hibernate_class_44.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/JodaTime29.java".
Successfully compiled, ignoring 1 'package R does not exist' errors for "/home/azmain/code_for_compilation_test/from-20-sample/Android02.java".
Failed to compile "/home/azmain/code_for_compilation_test/from-20-sample/xstream_class_23.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/xstream_class_3.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-20-sample/JodaTime08.java".
Successfully compiled "/home/azmain/code_for_compilation_test/f

In [5]:
# For Zero-shot Self-Consistency from 30 Sample

dir_type = "from-30-sample"
dir_path = f"/home/azmain/code_for_compilation_test/{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/zero-shot-{dir_type}/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/xstream_class_25.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/hibernate_class_24.java".
Failed to compile "/home/azmain/code_for_compilation_test/from-30-sample/hibernate_class_19.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/hibernate_class_44.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/JodaTime29.java".
Successfully compiled, ignoring 1 'package R does not exist' errors for "/home/azmain/code_for_compilation_test/from-30-sample/Android02.java".
Failed to compile "/home/azmain/code_for_compilation_test/from-30-sample/xstream_class_23.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/xstream_class_3.java".
Successfully compiled "/home/azmain/code_for_compilation_test/from-30-sample/JodaTime08.java".
Successfully compiled "/home/azmain/code_for_compilation_test/f

In [5]:
# For One-shot Self-Consistency from 10 Sample

dir_type = "one-shot"
dir_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/{dir_type}-compiled/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/xstream_class_25.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/hibernate_class_24.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/hibernate_class_19.java".
Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/hibernate_class_44.java".
Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/JodaTime29.java".
Successfully compiled, ignoring 1 'package R does not exist' errors for "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-one-shot/Android02.java".
Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java

In [6]:
# For Getting Error Message for StatType-SO

dir_path = "/home/azmain/GitHub Codes/so_dataset/"
jar_path = "/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_logs/compile_success/"
log_folder_fail = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_logs/compile_fail/"
compiled_folder_path = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_compiled-classes/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

Failed to compile "/home/azmain/GitHub Codes/so_dataset/xstream_class_25.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/hibernate_class_24.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/hibernate_class_19.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/hibernate_class_44.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/JodaTime29.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/Android02.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/xstream_class_23.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/xstream_class_3.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/JodaTime08.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/Class_19.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/Class_1.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/Android10.java".
Failed to compile "/home/azmain/GitHub Codes/so_dataset/hibernate_class_22.jav